In [5]:
import json
from s2ag_corpus.sql import INSERT_PAPER_SQL

base_dir = '/media/romilly/ss-corpus/2024-04-02/papers'

In [20]:
from s2ag_corpus.database_catalogue import test_connection

connection = test_connection()

def delete_papers_from_test_db():
    with connection.cursor() as cursor:
        cursor.execute('DELETE FROM papers')
        connection.commit()

In [21]:
delete_papers_from_test_db()
    

# I'm going to delete the rows and time an insertion of 10000 papers

In [8]:
connection.rollback()

In [22]:
with open(base_dir+'/first10000papers') as f:
    with connection.cursor() as cursor:
        for line in f:
            line = line.strip()
            # print(line)
            lj = json.loads(line)
            corpus_id = lj['corpusid']
            sha = lj['url'][-40:]
            cursor.execute(INSERT_PAPER_SQL, (corpus_id, sha, line))
            connection.commit()
    

In [23]:
delete_papers_from_test_db()

In [24]:
with open(base_dir+'/first10000papers') as f:
    with connection.cursor() as cursor:
        for line in f:
            line = line.strip()
            # print(line)
            lj = json.loads(line)
            corpus_id = lj['corpusid']
            sha = lj['url'][-40:]
            cursor.execute(INSERT_PAPER_SQL, (corpus_id, sha, line))
    connection.commit()

In [25]:
delete_papers_from_test_db()

In [26]:
from s2ag_corpus.database_catalogue import CorpusDatabaseCatalogue
catalogue = CorpusDatabaseCatalogue(test_connection())

In [27]:
with open(base_dir + '/first10000papers') as f:
    jason_dictionaries = [(line, json.loads(line)) for line in f.readlines()]
    records = [(jd['corpusid'], jd['url'][-40:], line) for  line, jd in jason_dictionaries]
catalogue.upsert(INSERT_PAPER_SQL, records)